In [1]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image

# Base severity weights

In [2]:
DISEASE_BASE_SEVERITY = {
    "nodisease": 0.0,
    "rust": 0.8,
    "miner": 0.6,
    "phoma": 0.7
}

# Image prediction

In [3]:
def predict_disease(model, class_indices, image_path, img_size=(128, 128)):
    img = image.load_img(image_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0

    preds = model.predict(img_array)
    class_idx = np.argmax(preds)
    confidence = float(np.max(preds))

    idx_to_class = {v: k for k, v in class_indices.items()}
    predicted_class = idx_to_class[class_idx]

    return predicted_class, confidence

# CNN-based severity

In [4]:
def cnn_severity(predicted_class, confidence):
    base = DISEASE_BASE_SEVERITY.get(predicted_class, 0.0)
    severity = base * confidence
    return round(severity, 3)

# Rust area-based severity

In [5]:
def rust_area_severity(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return 0.0

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    lower = np.array([5, 100, 100])
    upper = np.array([20, 255, 255])
    mask = cv2.inRange(hsv, lower, upper)

    rust_pixels = np.count_nonzero(mask)
    total_pixels = mask.size

    severity = rust_pixels / total_pixels
    return round(float(severity), 3)

# Final severity score

In [6]:
def compute_final_severity(model, class_indices, image_path, img_size=(128, 128)):
    disease, confidence = predict_disease(
        model, class_indices, image_path, img_size
    )

    cnn_score = cnn_severity(disease, confidence)

    # Apply area severity only for rust
    if disease == "rust":
        area_score = rust_area_severity(image_path)
        final_score = 0.5 * cnn_score + 0.5 * area_score
    else:
        final_score = cnn_score

    return {
        "disease": disease,
        "confidence": round(confidence, 3),
        "severity_score": round(final_score, 3)
    }

In [8]:
%%writefile severity_module.py
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image

# Disease Base Severity (Domain Knowledge)
# Each disease does not cause the same level of damage.
# So we assign a base severity weight based on agricultural impact:

DISEASE_BASE_SEVERITY = {
    "nodisease": 0.0,
    "rust": 0.8,
    "miner": 0.6,
    "phoma": 0.7
}

def predict_disease(model, class_indices, image_path, img_size=(128, 128)):
    img = image.load_img(image_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0

    preds = model.predict(img_array)
    class_idx = np.argmax(preds)
    confidence = float(np.max(preds))

    idx_to_class = {v: k for k, v in class_indices.items()}
    predicted_class = idx_to_class[class_idx]

    return predicted_class, confidence

def cnn_severity(predicted_class, confidence):
    base = DISEASE_BASE_SEVERITY.get(predicted_class, 0.0)
    return round(base * confidence, 3)

def rust_area_severity(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return 0.0

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower = np.array([5, 100, 100])
    upper = np.array([20, 255, 255])
    mask = cv2.inRange(hsv, lower, upper)

    rust_pixels = np.count_nonzero(mask)
    total_pixels = mask.size

    return round(rust_pixels / total_pixels, 3)

def compute_final_severity(model, class_indices, image_path, img_size=(128, 128)):
    disease, confidence = predict_disease(
        model, class_indices, image_path, img_size
    )

    cnn_score = cnn_severity(disease, confidence)

    if disease == "rust":
        area_score = rust_area_severity(image_path)
        final_score = 0.5 * cnn_score + 0.5 * area_score
    else:
        final_score = cnn_score

    return {
        "disease": disease,
        "confidence": round(confidence, 3),
        "severity_score": round(final_score, 3)
    }


Writing severity_module.py


# use

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [9]:
from tensorflow.keras.models import load_model
import json
from severity_module import compute_final_severity

model_path = '/content/drive/MyDrive/Coffee/Dev/C1/leaf disease/coffee_l_disease_classifier.keras'
class_indices_path = '/content/drive/MyDrive/Coffee/Dev/C1/leaf disease/class_indices_coffee_l_disease.json'
test_image_path = '/content/drive/MyDrive/Coffee/Dev/C1/leaf disease/test/756.jpg'

model = load_model(model_path)

with open(class_indices_path, 'r') as f:
    class_indices = json.load(f)

result = compute_final_severity(
    model=model,
    class_indices=class_indices,
    image_path=test_image_path
)

print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
{'disease': 'phoma', 'confidence': 0.673, 'severity_score': 0.471}
